In [3]:
!git commit -m "prediction age is completed 90%"

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 146 (delta 6), reused 5 (delta 3), pack-reused 134
Receiving objects: 100% (146/146), 124.84 MiB | 1.20 MiB/s, done.
Resolving deltas: 100% (46/46), completed with 1 local object.
From github.com:NikhilPandeyOfficial/face-analyser-generator
 * branch            develop    -> FETCH_HEAD
   75d61fd..e328516  develop    -> origin/develop
Updating 75d61fd..e328516
Fast-forward
 README.md                                          |  25 +
 dataset-extractor/.gitignore                       |  34 ++
 dataset-extractor/age_dataset/adult/dataset.csv    | 501 +++++++++++++++++++++
 dataset-extractor/age_dataset/adult/images.zip     | Bin 0 -> 4072334 bytes
 dataset-extractor/age_dataset/child/dataset.csv    | 501 +++++++++++++++++++++
 dataset-extractor/age_dataset/child/images.zip     | Bin 0 -> 3981316 bytes
 dataset-extractor/age_dataset/elderly/data

In [4]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from keras.utils import np_utils

In [6]:
dataset = pd.read_csv('dataset_200.csv')

In [7]:
s = str(dataset['image'].values[0])
print(s)

../../age_detection/age_gender_race/65_0_0_20170103183632050.jpg.chip.jpg


In [8]:
gray = cv2.cvtColor(cv2.imread(s), cv2.COLOR_BGR2GRAY)
gray.shape

(200, 200)

In [9]:
X = dataset.drop(columns=['target_age','target_gender','target_race','age','Unnamed: 0'])
y = dataset.drop(columns=['image','target_gender','target_race','age','Unnamed: 0'])
X

,image
0,../../age_detection/age_gender_race/65_0_0_201...
1,../../age_detection/age_gender_race/71_0_0_201...
2,../../age_detection/age_gender_race/38_0_1_201...
3,../../age_detection/age_gender_race/16_0_0_201...
4,../../age_detection/age_gender_race/1_1_4_2017...
...,...
23703,../../age_detection/age_gender_race/35_0_0_201...
23704,../../age_detection/age_gender_race/35_1_0_201...
23705,../../age_detection/age_gender_race/26_0_0_201...
23706,../../age_detection/age_gender_race/55_0_0_201...


In [10]:
print(X.shape,y.shape)

(23708, 1) (23708, 1)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [12]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [13]:
from tqdm import tqdm

In [14]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image']):
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face =cv2.resize(face, (200,200))
        face = np.expand_dims(face, axis=2)
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

In [15]:
X_train = get_data_from_local(X_train)
X_test = get_data_from_local(X_test)

  2%|▏         | 343/22522 [00:07<07:49, 47.22it/s]


KeyboardInterrupt: 

In [13]:
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
print(y_train.shape,y_test.shape,X_train.shape,X_test.shape)

(22522, 7) (1186, 7) (22522, 200, 200, 1) (1186, 200, 200, 1)


In [14]:
final_cnn = Sequential()
final_cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))  
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))
final_cnn.add(GlobalAveragePooling2D())

final_cnn.add(Dense(132, activation='relu'))

final_cnn.add(Dense(7, activation='softmax'))

final_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
average_pooling2d (AveragePo (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
average_pooling2d_2 (Average (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 256)       2

In [15]:
final_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Creating a ModelCheckpoint callback object to save the model according to the value of val_accuracy.

checkpoint = ModelCheckpoint(filepath=f"final_cnn_model_checkpoint.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

In [ ]:
# Fitting the above created CNN model.

final_cnn_history = final_cnn.fit(x=X_train,
                                  y=y_train,
                                  batch_size=512,
                                  validation_data=(X_test,y_test),
                                  epochs=60,
                                  callbacks=[checkpoint],
                                  shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                                 )